In [2]:
import numpy as np
import gym
from gym.spaces import Discrete, Box
# %tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import random
import gym
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class DeterministicDiscreteActionLinearPolicy(object):

    def __init__(self, theta, ob_space, ac_space):
        """
        dim_ob: dimension of observations
        n_actions: number of actions
        theta: flat vector of parameters
        """
        dim_ob = ob_space.shape[0]
        n_actions = ac_space.n
        assert len(theta) == (dim_ob + 1) * n_actions
        self.W = theta[0 : dim_ob * n_actions].reshape(dim_ob, n_actions)
        self.b = theta[dim_ob * n_actions : None].reshape(1, n_actions)

    def act(self, ob):
        """
        """
        y = ob.dot(self.W) + self.b
        a = y.argmax()
        return a

In [4]:
def do_episode(policy, env, num_steps, discount=1.0, render=False):
    disc_total_rew = 0
    ob = env.reset()
    for t in range(num_steps):
        a = policy.act(ob)
        (ob, reward, done, _info) = env.step(a)
        disc_total_rew += reward * discount**t
        if render and t%3==0:
            env.render()
        if done: break
    return disc_total_rew

env = None
def noisy_evaluation(theta, discount=1.0):
    policy = make_policy(theta)
    reward = do_episode(policy, env, num_steps, discount)
    return reward

def make_policy(theta):
    return DeterministicDiscreteActionLinearPolicy(theta, env.observation_space, env.action_space)

In [5]:
# Task settings:
env = gym.make('CartPole-v0') # Change as needed
num_steps = 200 # maximum length of episode
# env.monitor.start('/tmp/cartpole-experiment-1')

# Alg settings:
n_iter = 20 # number of iterations of CEM
batch_size = 25 # number of samples per batch
elite_frac = 0.2 # fraction of samples used as elite set
n_elite = int(batch_size * elite_frac)
extra_std = 2.0
extra_decay_time = 10
dim_theta = (env.observation_space.shape[0]+1) * env.action_space.n

# Initialize mean and standard deviation
theta_mean = np.zeros(dim_theta)
theta_std = np.ones(dim_theta)

## CEM

In [6]:
# Now, for the algorithm
for itr in range(n_iter):
    # Sample parameter vectors
    extra_cov = max(1.0 - itr / extra_decay_time, 0) * extra_std**2
    thetas = np.random.multivariate_normal(mean=theta_mean, 
                                           cov=np.diag(np.array(theta_std**2) + extra_cov), 
                                           size=batch_size)
    #rewards = np.array(map(noisy_evaluation, thetas))
    rewards = np.array([noisy_evaluation(theta) for theta in thetas])

    # Get elite parameters
    elite_inds = rewards.argsort()[-n_elite:]
    elite_thetas = thetas[elite_inds]

    # Update theta_mean, theta_std
    theta_mean = elite_thetas.mean(axis=0)
    theta_std = elite_thetas.std(axis=0)
    print("iteration {} mean {} max {}".format(itr, np.mean(rewards), np.max(rewards)))
    do_episode(make_policy(theta_mean), env, num_steps, discount=0.90, render=True)

#     print("theta mean", theta_mean)
#     print("theta std", theta_std)


iteration 0 mean 16.6 max 114.0
iteration 1 mean 21.28 max 81.0
iteration 2 mean 36.48 max 129.0
iteration 3 mean 54.4 max 138.0
iteration 4 mean 79.16 max 200.0
iteration 5 mean 67.68 max 200.0
iteration 6 mean 115.68 max 200.0
iteration 7 mean 150.48 max 200.0
iteration 8 mean 191.8 max 200.0
iteration 9 mean 184.28 max 200.0
iteration 10 mean 184.32 max 200.0
iteration 11 mean 162.0 max 200.0
iteration 12 mean 200.0 max 200.0
iteration 13 mean 199.52 max 200.0
iteration 14 mean 200.0 max 200.0
iteration 15 mean 200.0 max 200.0
iteration 16 mean 200.0 max 200.0
iteration 17 mean 200.0 max 200.0
iteration 18 mean 200.0 max 200.0
iteration 19 mean 200.0 max 200.0


## Random Shooting


In [7]:
thetas = np.random.uniform(size = batch_size)
thetas = np.random.multivariate_normal(mean=np.zeros(dim_theta), 
                                           cov=np.diag(np.ones(dim_theta)), 
                                           size=batch_size)
rewards = np.array([noisy_evaluation(theta) for theta in thetas])
elite_ind = rewards.argmax()
elite_theta = thetas[elite_ind]

print("best theta:", elite_theta)
print("reward:", rewards[elite_ind])

best theta: [ 0.13538736  0.13866565  0.00339023 -1.1278153  -1.27884826 -0.73940764
  0.38156817  0.80112322  0.52504811  0.4836304 ]
reward: 69.0


## Gradient Descent

In [10]:
def policy_gradient():
    with tf.variable_scope("policy", reuse=tf.AUTO_REUSE):
        params = tf.get_variable("policy_parameters",[4,2])
        state = tf.placeholder("float",[None,4])
        actions = tf.placeholder("float",[None,2])
        advantages = tf.placeholder("float",[None,1])
        linear = tf.matmul(state,params)
        probabilities = tf.nn.softmax(linear)
        good_probabilities = tf.reduce_sum(tf.multiply(probabilities, actions),reduction_indices=[1])
        eligibility = tf.log(good_probabilities) * advantages
        loss = -tf.reduce_sum(eligibility)
        optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
        return probabilities, state, actions, advantages, optimizer

def value_gradient():
    with tf.variable_scope("value", reuse=tf.AUTO_REUSE):
        state = tf.placeholder("float",[None,4])
        newvals = tf.placeholder("float",[None,1])
        w1 = tf.get_variable("w1",[4,10])
        b1 = tf.get_variable("b1",[10])
        h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
        w2 = tf.get_variable("w2",[10,1])
        b2 = tf.get_variable("b2",[1])
        calculated = tf.matmul(h1,w2) + b2
        diffs = calculated - newvals
        loss = tf.nn.l2_loss(diffs)
        optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)
        return calculated, state, newvals, optimizer, loss

def run_episode(env, policy_grad, value_grad, sess, render=False):
    pl_calculated, pl_state, pl_actions, pl_advantages, pl_optimizer = policy_grad
    vl_calculated, vl_state, vl_newvals, vl_optimizer, vl_loss = value_grad
    observation = env.reset()
    totalreward = 0
    states = []
    actions = []
    advantages = []
    transitions = []
    update_vals = []


    for t in range(200):
        # calculate policy
        obs_vector = np.expand_dims(observation, axis=0)
        probs = sess.run(pl_calculated,feed_dict={pl_state: obs_vector})
        action = 0 if random.uniform(0,1) < probs[0][0] else 1
        # record the transition
        states.append(observation)
        actionblank = np.zeros(2)
        actionblank[action] = 1
        actions.append(actionblank)
        # take the action in the environment
        old_observation = observation
        observation, reward, done, info = env.step(action)
        transitions.append((old_observation, action, reward))
        totalreward += reward
        
        if t % 3 == 0 and render:
            env.render()

        if done:
            break
    for index, trans in enumerate(transitions):
        obs, action, reward = trans

        # calculate discounted monte-carlo return
        future_reward = 0
        future_transitions = len(transitions) - index
        decrease = 1
        for index2 in range(future_transitions):
            future_reward += transitions[(index2) + index][2] * decrease
            decrease = decrease * 0.97
        obs_vector = np.expand_dims(obs, axis=0)
        currentval = sess.run(vl_calculated,feed_dict={vl_state: obs_vector})[0][0]

        # advantage: how much better was this action than normal
        advantages.append(future_reward - currentval)

        # update the value function towards new return
        update_vals.append(future_reward)

    # update value function
    update_vals_vector = np.expand_dims(update_vals, axis=1)
    sess.run(vl_optimizer, feed_dict={vl_state: states, vl_newvals: update_vals_vector})
    # real_vl_loss = sess.run(vl_loss, feed_dict={vl_state: states, vl_newvals: update_vals_vector})

    advantages_vector = np.expand_dims(advantages, axis=1)
    sess.run(pl_optimizer, feed_dict={pl_state: states, pl_advantages: advantages_vector, pl_actions: actions})

    return totalreward

In [12]:
env = gym.make('CartPole-v0')
policy_grad = policy_gradient()
value_grad = value_gradient()
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
t = 0
for i in range(2000):
    reward = run_episode(env, policy_grad, value_grad, sess, render=True)
    t += reward
    if i % 100 == 0:
        print(reward)

print(t / 1000)

17.0
19.0
18.0
69.0
200.0
200.0
200.0
185.0


KeyboardInterrupt: 